## Model Building: Predict Musuems' Ratings  
Author: Anne Chen  
2016

In [1]:
import numpy as np
import pandas as pd
from fancyimpute import KNN

Using Theano backend.


In [2]:
m_dum = pd.read_csv("dummified_df.csv")
m_dum.head()

,Unnamed: 0,Address,Description,FeatureCount,Fee,Langtitude,Latitude,LengthOfVisit,MuseumName,PhoneNum,...,descri_pol,descri_sub,LengthOfVisit__1-2 hours,LengthOfVisit__2-3 hours,LengthOfVisit__<1 hour,LengthOfVisit__More than 3 hours,Country__Other,Country__USA,Fee__No,Fee__Yes
0,0,"555 Pennsylvania Ave NW, Washington DC, DC 200...",Find out for yourself why everyone is calling ...,3.0,Yes,-77.019235,38.893138,2-3 hours,Newseum,+1 888-639-7386,...,0.490000,0.506667,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1,4,"1000 5th Ave, New York City, NY 10028-0198",At New York City's most visited museum and att...,12.0,Yes,-73.962928,40.779166,2-3 hours,The Metropolitan Museum of Art,1 212-535-7710,...,0.318182,0.477273,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,5,"945 Magazine Street, New Orleans, LA 70130-3813","Founded by historian and author, Stephen Ambro...",11.0,NaN,-90.070086,29.943004,NaN,The National WWII Museum,+1 504-528-1944,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,6,"2001 N Colorado Blvd, Denver, CO 80205-5798",The Denver Museum of Nature & Science is the R...,0.0,NaN,-104.941020,39.769189,NaN,Denver Museum of Nature & Science,303-370-6000,...,0.100000,0.400000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,7,"111 S Michigan Ave, Chicago, IL 60603-6488","This Classical Renaissance structure, guarded ...",5.0,Yes,-87.623724,41.879547,More than 3 hours,Art Institute of Chicago,312 443 3600,...,0.300000,0.337500,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0


In [4]:
col_idx_df = pd.DataFrame({'colname':m_dum.columns.values})
col_idx_df
# idx to drop: [0,1,2,4,5,6,7,8,9,14,15]

,colname
0,Unnamed: 0
1,Address
2,Description
3,FeatureCount
4,Fee
5,Langtitude
6,Latitude
7,LengthOfVisit
8,MuseumName
9,PhoneNum


In [5]:
# drop columns that are irrevelant to modeling 
# and original categorical features which had beend dummified
# columns of interest: column index 11 'Rating' OR column index 64 'PreciseRating'
# dataframe w/ displayed rating 
idx_to_drop = [0,1,2,4,5,6,7,8,9,14,15]
museum_d = m_dum.drop(m_dum.columns[idx_to_drop + [64]], axis=1)
# dataframe w/ precise rating
museum_p = m_dum.drop(m_dum.columns[idx_to_drop + [11]], axis=1)

In [6]:
museum_d.describe() # descri_pol and descri_sub have NAs

/Users/annecool37/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,FeatureCount,Rank,Rating,ReviewCount,TotalThingsToDo,RankPercentage,Ancient Ruins,Architectural Buildings,Arenas & Stadiums,Art Galleries,...,descri_pol,descri_sub,LengthOfVisit__1-2 hours,LengthOfVisit__2-3 hours,LengthOfVisit__<1 hour,LengthOfVisit__More than 3 hours,Country__Other,Country__USA,Fee__No,Fee__Yes
count,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,...,1161.000000,1161.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000
mean,1.007486,16.555209,4.426700,1540.467249,243.429195,9.297117,0.001248,0.006862,0.001871,0.006862,...,0.179419,0.397966,0.149719,0.123518,0.021834,0.054273,0.388022,0.611978,0.032439,0.115409
std,2.149806,28.900707,0.285016,3906.633803,308.115561,10.134265,0.035311,0.082579,0.043234,0.082579,...,0.185510,0.221829,0.356907,0.329134,0.146187,0.226627,0.487452,0.487452,0.177219,0.319614
min,0.000000,1.000000,2.500000,32.000000,1.000000,0.069300,0.000000,0.000000,0.000000,0.000000,...,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,4.500000,213.000000,48.000000,2.941176,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,6.000000,4.500000,510.000000,142.000000,6.250000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,1.000000,19.000000,4.500000,1257.000000,314.500000,12.500000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,27.000000,397.000000,5.000000,63112.000000,2279.000000,100.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
museum_d.describe().to_csv('summary_before_imputation.csv')

In [7]:
museum_d.columns

Index([u'FeatureCount', u'Rank', u'Rating', u'ReviewCount', u'TotalThingsToDo',
       u'RankPercentage', u'Ancient Ruins', u'Architectural Buildings',
       u'Arenas & Stadiums', u'Art Galleries',
       ...
       u'descri_pol', u'descri_sub', u'LengthOfVisit__1-2 hours ',
       u'LengthOfVisit__2-3 hours ', u'LengthOfVisit__<1 hour ',
       u'LengthOfVisit__More than 3 hours ', u'Country__Other',
       u'Country__USA', u'Fee__No ', u'Fee__Yes '],
      dtype='object', length=220)

### Exploring Different Models - Multilinear Regression and Random Forest

In [7]:
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

In [8]:
# m_x = museum_d.drop(museum_d.columns[[2]], axis=1) # drop rating
# m_y = museum_d['Rating']
# m_x = df.drop(df.columns[[193]], axis=1) # drop PreciseRating
# m_y = df['PreciseRating']
def explore_model(df, col_name_to_pred, col_idx_to_drop):
    np.random.seed(0)
    m_x = df.drop(df.columns[[col_idx_to_drop]], axis=1) 
    m_y = df[col_name_to_pred]
    n_samples = m_x.shape[0]
    n_features = m_x.shape[1]
    print 'number of samples:', n_samples
    print 'number of features', n_features
    
    # subset training and testing
    x_train, x_test, y_train, y_test = train_test_split(m_x, m_y, test_size=0.2, random_state = 0)
    
    # linear regression
    ols = linear_model.LinearRegression()
    ols.fit(x_train, y_train)

    print 'multilinear score:', ols.score(x_train, y_train)
    predicted = ols.predict(x_test)
    print 'multilinear MSE:', np.mean((predicted - y_test)**2)

    # store coefficients
    coef_linear = pd.DataFrame({'feature':m_x.columns, 'coef': ols.coef_})
    coef_linear = coef_linear.sort_values(by = 'coef', axis=0, ascending = False)
    
    # random forest (regression)
    rf = RandomForestRegressor()
    rf.fit(x_train, y_train)
    print 'random forest (regression) score:', rf.score(x_train, y_train)
    predicted = rf.predict(x_test)
    print 'random forest (regression) MSE:', np.mean((predicted - y_test)**2)
    
    # random forest (classification
    # convert predictor from numeric into string
    y_train = [str(i) for i in y_train]
    y_test = [str(i) for i in y_test]        
    np.random.seed(0)
    rf_c = RandomForestClassifier()
    rf_c.fit(x_train, y_train)
    predicted = rf_c.predict(x_test)
    err = 1 - np.mean(predicted == y_test)
    print 'random forest (classification) score:', rf_c.score(x_train, y_train)
    predicted = rf_c.predict(x_test)
    print 'random forest (classification) error:', err
    
    # feature importance
    feature_imprtance = zip(m_x.columns, rf.feature_importances_)
    dtype = [('feature', 'S10'), ('importance', 'float')]
    feature_imprtance = np.array(feature_imprtance, dtype = dtype)
    feature_sort = np.sort(feature_imprtance, order='importance')[::-1]
    print 'RF regression'
    print 'top 10 important features:'
    print feature_sort[0:10]
    
    feature_imprtance_c = zip(m_x.columns, rf_c.feature_importances_)
    dtype = [('feature', 'S10'), ('importance', 'float')]
    feature_imprtance_c = np.array(feature_imprtance_c, dtype = dtype)
    feature_sort_c = np.sort(feature_imprtance_c, order='importance')[::-1]
    print 'RF Classification'
    print 'top 10 important features:'
    print feature_sort_c[0:10]
    
    return coef_linear, feature_sort, feature_sort_c 

In [9]:
def cv_rf_class(k_grid, df, col_name_to_pred, col_idx_to_drop):
    err_lst = []
    score_lst = []
    length = df.shape[0]
    for k_val in k_grid:
        X_filled_knn = KNN(k = k_val).complete(df)
        knn_imputed_md = pd.DataFrame(data = X_filled_knn,
                                      index= range(0,length),
                                      columns = df.columns)
        
        m_x = knn_imputed_md.drop(knn_imputed_md.columns[[col_idx_to_drop]], axis=1)        
        m_y = [str(i) for i in knn_imputed_md[col_name_to_pred]]
        x_train, x_test, y_train, y_test = train_test_split(m_x, m_y, test_size=0.2, random_state = 0)        
        np.random.seed(0)
        rf = RandomForestClassifier()
        rf.fit(x_train, y_train)
        predicted = rf.predict(x_test)
        score_lst.append(rf.score(x_train, y_train))
        err_lst.append(1 - np.mean(predicted == y_test))
    result_df = pd.DataFrame({'k':k_grid, 'score':score_lst, 'error':err_lst})  
    result_df = result_df.sort_values(by = 'error', axis=0)
    return result_df

def cv_rf_regres(k_grid, df, col_name_to_pred, col_idx_to_drop):
    mse_lst = []
    score_lst = []
    length = df.shape[0]
    for k_val in k_grid:
        X_filled_knn = KNN(k = k_val).complete(df)
        knn_imputed_md = pd.DataFrame(data = X_filled_knn,
                                      index= range(0,length),
                                      columns = df.columns)
        m_x = knn_imputed_md.drop(knn_imputed_md.columns[[col_idx_to_drop]], axis=1)
        m_y = knn_imputed_md[col_name_to_pred]
        np.random.seed(0)
        rf = RandomForestRegressor()            
        x_train, x_test, y_train, y_test = train_test_split(m_x, m_y, test_size=0.2, random_state = 0)
        rf.fit(x_train, y_train)
        predicted = rf.predict(x_test)
        score_lst.append(rf.score(x_train, y_train))
        mse_lst.append(np.mean((predicted - y_test)**2))
    result_df = pd.DataFrame({'k':k_grid, 'score':score_lst, 'mse':mse_lst})  
    result_df = result_df.sort_values(by = 'mse', axis=0)
    return result_df

## Predict Rating
missing values are handled with mean or knn imputation

#### Mean Imputation

In [10]:
# imput NA with mean value
mean_impute_d = museum_d.copy(deep = True)
mean_impute_p = museum_p.copy(deep = True)

mean_impute_d['descri_pol'] = museum_d['descri_pol'].fillna(np.mean(museum_d['descri_pol']))
mean_impute_d['descri_sub'] = museum_d['descri_sub'].fillna(np.mean(museum_d['descri_sub']))

mean_impute_p['descri_pol'] = museum_p['descri_pol'].fillna(np.mean(museum_p['descri_pol']))
mean_impute_p['descri_sub'] = museum_p['descri_sub'].fillna(np.mean(museum_p['descri_sub']))

In [13]:
(museum_p.columns == 'PreciseRating').tolist().index(True)

193

In [14]:
(museum_d.columns == 'Rating').tolist().index(True)

2

In [11]:
explore_model(mean_impute_d,'Rating',[2, 193])

number of samples: 1603
number of features 218
multilinear score: 0.473653183562
multilinear MSE: 0.0609866180954
random forest (regression) score: 0.876169452672
random forest (regression) MSE: 0.0568535825545
random forest (classification) score: 0.983619344774
random forest (classification) error: 0.274143302181
RF regression
top 10 important features:
[('Rank', 0.24679201964963987) ('RankPercen', 0.10720257468757007)
 ('review_pol', 0.046074096716786826) ('quote_sub_', 0.04212617145547866)
 ('review_pol', 0.02964226445210446) ('ReviewCoun', 0.02958528532914046)
 ('Friends_Co', 0.026058881882098005) ('Couples_Co', 0.0227666079291552)
 ('quote_pol_', 0.02177004983768878) ('quote_pol_', 0.02066406826032283)]
RF Classification
top 10 important features:
[('RankPercen', 0.06266225594213995) ('Rank', 0.04739976579805334)
 ('review_pol', 0.03947451085555946) ('TotalThing', 0.0381469799115917)
 ('quote_sub_', 0.037377531721634195) ('review_pol', 0.031844117807274065)
 ('review_pol', 0.0307

(         coef                      feature
 192  1.015978                quote_pol_var
 45   0.592839                        Mines
 7    0.483363            Arenas & Stadiums
 66   0.413150         Scenic Walking Areas
 34   0.356770       Historic Walking Areas
 196  0.346120                quote_sub_var
 68   0.323701  Self-Guided Tours & Rentals
 15   0.321999                      Bridges
 201  0.302699              review_pol_mean
 5    0.280447                Ancient Ruins
 195  0.201537                quote_pol_min
 83   0.192732                Walking Tours
 63   0.152497                Private Tours
 32   0.151410         Government Buildings
 39   0.151267                    Libraries
 11   0.140921                      Ballets
 19   0.117721                   City Tours
 84   0.114223         Wineries & Vineyards
 27   0.114223                 Food & Drink
 23   0.109787               Cultural Tours
 119  0.107114           amazing collection
 54   0.102102                  

#### Predict Rating (missing values handled by KNN imputation)

In [ ]:
col_name_to_pred = 'Rating'
col_idx_to_drop = [2, 193]
length = museum_d.shape[0]
# KNN imputation
X_filled_knn = KNN(k = 40).complete(museum_d)
knn_imputed_md = pd.DataFrame(data = X_filled_knn,
                              index= range(0,length),
                              columns = museum_d.columns)

In [15]:
# random forest (regression)
m_x = knn_imputed_md.drop(knn_imputed_md.columns[[col_idx_to_drop]], axis=1)
m_y = knn_imputed_md[col_name_to_pred]
np.random.seed(0)
rf = RandomForestRegressor()            
x_train, x_test, y_train, y_test = train_test_split(m_x, m_y, test_size=0.2, random_state = 0)
rf.fit(x_train, y_train)
predicted = rf.predict(x_test)
print 'rf regression score:', rf.score(x_train, y_train)
print 'rf regression MSE:', np.mean((predicted - y_test)**2)

# random forest (classification)
m_y = [str(i) for i in knn_imputed_md[col_name_to_pred]]
x_train, x_test, y_train, y_test = train_test_split(m_x, m_y, test_size=0.2, random_state = 0)        
np.random.seed(0)
rf_c = RandomForestClassifier()
rf_c.fit(x_train, y_train)
predicted = rf_c.predict(x_test)
print 'rf classification score:', rf_c.score(x_train, y_train)
print 'rf classification error:', 1 - np.mean(predicted == y_test)

rf regression score: 0.87402998432
rf regression MSE: 0.057468847352
rf classification score: 0.98127925117
rf classification error: 0.264797507788


In [14]:
feature_imprtance = zip(m_x.columns, rf.feature_importances_)
dtype = [('feature', 'S10'), ('importance', 'float')]
feature_imprtance = np.array(feature_imprtance, dtype = dtype)
feature_sort = np.sort(feature_imprtance, order='importance')[::-1]
print 'RF regression'
print 'top 10 important features:'
print feature_sort[0:10]

feature_imprtance_c = zip(m_x.columns, rf_c.feature_importances_)
dtype = [('feature', 'S10'), ('importance', 'float')]
feature_imprtance_c = np.array(feature_imprtance_c, dtype = dtype)
feature_sort_c = np.sort(feature_imprtance_c, order='importance')[::-1]
print 'RF Classification'
print 'top 10 important features:'
print feature_sort_c[0:10]

RF regression
top 10 important features:
[('Rank', 0.2469439007840352) ('RankPercen', 0.10496319925830844)
 ('review_pol', 0.04839848858406851) ('quote_sub_', 0.039876003593301124)
 ('ReviewCoun', 0.028345457764503517) ('review_pol', 0.027778905434448747)
 ('Friends_Co', 0.024196828498994623) ('Couples_Co', 0.022909676903077585)
 ('quote_pol_', 0.022135199678854335) ('quote_pol_', 0.020598187760635107)]
RF Classification
top 10 important features:
[('RankPercen', 0.07461934071254318) ('Rank', 0.049009168423989065)
 ('TotalThing', 0.04332905945954477) ('review_pol', 0.03728393538958517)
 ('review_pol', 0.03493304780238371) ('quote_sub_', 0.03355374256483755)
 ('quote_sub_', 0.02837201106312263) ('review_sub', 0.028332846575792915)
 ('review_sub', 0.027707107246642902) ('review_sub', 0.027304116140127965)]


In [28]:
pd.DataFrame({'colname':museum_p.columns.values})

,colname
0,FeatureCount
1,Rank
2,ReviewCount
3,TotalThingsToDo
4,RankPercentage
5,Ancient Ruins
6,Architectural Buildings
7,Arenas & Stadiums
8,Art Galleries
9,Art Museums
